# Introduccion a Spark SQL

![title](images\spark-sql.png)

Es la interfaz de Spark para trabajar con datos estructurados y semiestructurados


- Datos estructurados son todos aquellos datos que presentan un esquema, es decir, existe un conjunto de campos para cada registro.
- Spark SQL proporciona un conjunto de datos abstractos que simplifica el trabajo con conjuntos de datos estructurados. El conjunto de datos es similar a las tablas de una base de datos relacional.
- Cada vez mas el flujo de trabajo de Spark se mueve hacia Spark SQL
- El principal objetivo es trabajar con datos estructurados.
- Un conjunto de datos tiene un esquema predeterminado, y esto permite que Spark almacene informacion de una manera mas eficiente y pueda ejecutar consultas SQL sobre estos datos utilizando comandos SQL


### Conceptos importantes de Spark SQL

#### 1. Dataframes
#### 2. Conjuntos de datos (Datasets)

### Dataframes

- Spark SQL introduce una abstraccion de datos tabulares llamada **Dataframe** a partir de la version 1.3.
- Un Dataframe es una abstraccion de datos o un lenguaje especifico de dominio para trabajar con datos estructurados o semiestructurados
- Los Dataframes almacenan datos de una manera mas eficiente comparados con RDDs nativos, aprovechando su esquema
- Utiliza las capacidades inmutables, en memoria, resilentes, distribuidas y paralelas de un RDD y aplica una estructura llamada esquema a los datos, permitiendo asi que Spark administre esta estrucutra y solo pase datos entre nodos de una manera mucho mas eficiente en lugar de la serializacion de objetos en Java
- A diferencia de un RDD, la informacion es organizada en columnas, con nombres especificos similar a una tabla en una Base de datos Relacional


![title](images\dataframe.png)

### Conjuntos de Datos (Datasets)

El API de los conjuntos de datos, presente desde la version 1.6 de Spark proporciona:
- El familiar estilo de programacion orientado a objetos
- El Compile-Time y Type Safety de la API del RDD
- El beneficio de usar los sistemas para trabajar con datos estructurados

Un Conjunto de datos generalmente es un conjunto de datos estructurados, no necesesariamente una fila pero podria ser de tipo particular


![title](images\dataset-dataframe-rdd.png)


### Ejemplo:

In [11]:
import org.apache.spark.sql.SparkSession

val AGE_MIDPOINT = "age_midpoint"
val SALARY_MIDPOINT = "salary_midpoint"
val SALARY_MIDPOINT_BUCKET = "salary_midpoint_bucket"

val session = SparkSession.builder().appName("StackOverFlowSurvey").master("local[1]").getOrCreate()

val dataFrameReader = session.read

val responses = dataFrameReader
  .option("header", "true")
  .option("inferSchema", value = true)
  .csv("input/2016-stack-overflow-survey-responses.csv")

System.out.println("=== Print out schema ===")
responses.printSchema()

val responseWithSelectedColumns = responses.select("country", "occupation", AGE_MIDPOINT, SALARY_MIDPOINT)

System.out.println("=== Print the selected columns of the table ===")
responseWithSelectedColumns.show()

System.out.println("=== Print records where the response is from Afghanistan ===")
responseWithSelectedColumns.filter(responseWithSelectedColumns.col("country").===("Afghanistan")).show()

System.out.println("=== Print the count of occupations ===")
val groupedDataset = responseWithSelectedColumns.groupBy("occupation")
groupedDataset.count().show()

System.out.println("=== Print records with average mid age less than 20 ===")
responseWithSelectedColumns.filter(responseWithSelectedColumns.col(AGE_MIDPOINT) < 20).show()

System.out.println("=== Print the result by salary middle point in descending order ===")
responseWithSelectedColumns.orderBy(responseWithSelectedColumns.col(SALARY_MIDPOINT).desc).show()

System.out.println("=== Group by country and aggregate by average salary middle point ===")
val datasetGroupByCountry = responseWithSelectedColumns.groupBy("country")
datasetGroupByCountry.avg(SALARY_MIDPOINT).show()

val responseWithSalaryBucket = responses.withColumn(SALARY_MIDPOINT_BUCKET,
  responses.col(SALARY_MIDPOINT).divide(20000).cast("integer").multiply(20000))

System.out.println("=== With salary bucket column ===")
responseWithSalaryBucket.select(SALARY_MIDPOINT, SALARY_MIDPOINT_BUCKET).show()

System.out.println("=== Group by salary bucket ===")
responseWithSalaryBucket.groupBy(SALARY_MIDPOINT_BUCKET).count().orderBy(SALARY_MIDPOINT_BUCKET).show()

session.stop()


=== Print out schema ===
root
 |-- _c0: integer (nullable = true)
 |-- collector: string (nullable = true)
 |-- country: string (nullable = true)
 |-- un_subregion: string (nullable = true)
 |-- so_region: string (nullable = true)
 |-- age_range: string (nullable = true)
 |-- age_midpoint: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- self_identification: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- occupation_group: string (nullable = true)
 |-- experience_range: string (nullable = true)
 |-- experience_midpoint: double (nullable = true)
 |-- salary_range: string (nullable = true)
 |-- salary_midpoint: double (nullable = true)
 |-- big_mac_index: double (nullable = true)
 |-- tech_do: string (nullable = true)
 |-- tech_want: string (nullable = true)
 |-- aliens: string (nullable = true)
 |-- programming_ability: double (nullable = true)
 |-- employment_status: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- company


=== Group by country and aggregate by average salary middle point ===
+------------------+--------------------+
|           country|avg(salary_midpoint)|
+------------------+--------------------+
|           Germany|  46491.228070175435|
|       Afghanistan|             66250.0|
|          Cambodia|              5000.0|
|            France|  39648.760330578516|
|           Algeria|             30000.0|
|         Argentina|  27950.819672131147|
|           Belgium|   45989.01098901099|
|           Ecuador|             40000.0|
|           Albania|   8333.333333333334|
|           Finland|   45714.28571428572|
|           Bahamas|             95000.0|
|             China|             54687.5|
|           Belarus|             10000.0|
|             Chile|  41666.666666666664|
|           Croatia|  14166.666666666666|
|           Andorra|             40000.0|
|           Bolivia|              5000.0|
|           Denmark|   68768.65671641791|
|        Bangladesh|   7307.692307692308|
|Bosn

import org.apache.spark.sql.SparkSession
AGE_MIDPOINT: String = age_midpoint
SALARY_MIDPOINT: String = salary_midpoint
SALARY_MIDPOINT_BUCKET: String = salary_midpoint_bucket
session: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3a5cea33
dataFrameReader: org.apache.spark.sql.DataFrameReader = org.apache.spark.sql.DataFrameReader@57a32ccb
responses: org.apache.spark.sql.DataFrame = [_c0: int, collector: string ... 64 more fields]
responseWithSelectedColumns: org.apache.spark.sql.DataFrame = [country: string, occupation: string ... 2 more fields]
groupedDataset: org.apache.spark.sql.RelationalGroupedDataset = RelationalGroupedDataset: [grouping expressions: [occupation: string], value: [country: string, occupation: string ... 2 more fields], type: GroupBy]...

### Ejercicio:

Cree un programa Spark para leer los datos de la casa desde / RealEstate.csv, agrupe por ubicación, agregue el precio promedio por SQ Ft y ordene por precio promedio por SQ Ft.

El conjunto de datos de casas contiene una colección de listados de bienes inmuebles recientes en el condado de San Luis Obispo y
alrededor.

El conjunto de datos contiene los siguientes campos:
1. MLS: número de servicio de listado múltiple para la casa (identificación única).
2. Ubicación: ciudad / pueblo donde se encuentra la casa. La mayoría de las ubicaciones están en el condado de San Luis Obispo y norte del condado de Santa Bárbara (Santa Maria Orcutt, Lompoc, Guadalupe, Los Alamos), pero allí algunas ubicaciones fuera del área también.
3. Precio: el precio de listado más reciente de la casa (en dólares).
4. Dormitorios: número de dormitorios.
5. Baños: número de baños.
6. Tamaño: tamaño de la casa en pies cuadrados.
7. Precio / Pies Cuadrados: precio de la casa por pie cuadrado.
8. Estado: tipo de venta. Estos tipos están representados en el conjunto de datos: venta corta, ejecución hipotecaria y regular.

Cada campo está separado por comas.

### Solucion:

In [8]:
import org.apache.spark.sql.SparkSession

val PRICE_SQ_FT = "Price SQ Ft"

val session = SparkSession.builder().appName("HousePriceSolution").master("local[1]").getOrCreate()

val realEstate = session.read
      .option("header", "true")
      .option("inferSchema", value = true)
      .csv("input/RealEstate.csv")

realEstate.groupBy("Location")
      .avg(PRICE_SQ_FT)
      .orderBy("avg(Price SQ Ft)")
      .show()

+-------------------+------------------+
|           Location|  avg(Price SQ Ft)|
+-------------------+------------------+
|         New Cuyama|             34.05|
|        Bakersfield|             69.69|
|          King City| 71.51333333333334|
|         Greenfield|             91.58|
|    Santa Margarita|             95.38|
|            Soledad|102.69333333333333|
|        Out Of Area|116.23333333333333|
|          Guadalupe|           120.175|
|           Coalinga|124.34285714285714|
| Santa Maria-Orcutt|147.58871698113194|
|             Lompoc|             149.9|
|             Lompoc|159.87115384615387|
|         San Miguel|163.16071428571425|
|            Bradley|            166.81|
|            Creston|            181.76|
| Santa Maria-Orcutt|183.03692307692307|
|             Nipomo|187.92333333333332|
|        Paso Robles|191.17752941176474|
|         Los Alamos|191.99333333333334|
|            Solvang|           193.305|
+-------------------+------------------+
only showing top

import org.apache.spark.sql.SparkSession
PRICE_SQ_FT: String = Price SQ Ft
session: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3a5cea33
realEstate: org.apache.spark.sql.DataFrame = [MLS: int, Location: string ... 6 more fields]
